In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import torch

In [2]:
class Layer(torch.nn.Module):
    def __init__(self, size_in, size_out, activation):
        super(Layer,self).__init__()
        self.weights = torch.nn.Parameter(torch.randn(size_in, size_out,requires_grad=True))
        self.bias = torch.nn.Parameter(torch.randn(1,size_out,requires_grad=True))
        self.activation = activation
        
    def Forward(self, z_in):
        return self.activation(z_in@self.weights + self.bias)

In [3]:
forget = Layer(38,15, torch.nn.Sigmoid())
loss_func = torch.nn.MSELoss()
opt = torch.optim.Adam(forget.parameters())

In [4]:
x_in = torch.randn(1,38)
y = torch.rand(1,15)
print(forget.bias)


Parameter containing:
tensor([[ 0.9230, -1.3775,  0.0349,  0.8350,  0.3566,  0.0389, -1.3132, -0.7851,
         -0.8781, -0.8378, -0.6207,  1.2292,  0.0428,  1.5867, -0.5709]],
       requires_grad=True)


In [5]:
out = forget.Forward(x_in)
loss = loss_func(out,y)
loss.backward()
opt.step()
opt.zero_grad()
print(forget.bias)

Parameter containing:
tensor([[ 0.9220, -1.3765,  0.0339,  0.8340,  0.3576,  0.0379, -1.3122, -0.7861,
         -0.8791, -0.8388, -0.6216,  1.2285,  0.0438,  1.5877, -0.5719]],
       requires_grad=True)


## Recurrent Neuro Network 
It passes some information to forward layers.... and it takes some data and saves it and uses it again... it kinda creates a 

In [6]:
class RNN(torch.nn.Module):
    def __init__(self, size_in, size_out, size_mem):
        super(RNN,self).__init__()
        self.size_mem = size_mem
        self.mem_layer = Layer(size_in + size_mem, size_mem, torch.tanh)
        self.out_layer = Layer(size_mem, size_out, torch.sigmoid)
        
    def Forward(self, X):
        mem = torch.zeros(1,self.size_mem)
        y_hat = []
        for i in range(X.shape[0]):
            x_in = X[[i],:]
            z_in = torch.cat([x_in,mem],dim = 1)
            mem = self.mem_layer.Forward(x_in)
            y_hat.append(self.out_layer.Forward(men))
        return torch.cat(y_hat,dim = 0)

In [7]:
rnn = RNN(38,15,5)

loss_func = torch.nn.MSELoss()

opt = torch.optim.Adam(rnn.parameters())

In [8]:
opt

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [9]:
print(rnn.mem_layer.bias)

Parameter containing:
tensor([[ 1.1068,  0.3737, -1.1810, -0.3015,  0.5125]], requires_grad=True)


In [10]:
y_hat = rnn.Forward(x_in)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x38 and 43x5)

In [ ]:
loss = loss_func(y_hat,y)
loss.backward()
opt.step()
opt.zero_grad()
print()
print(rnn.mem_layer.bias)

# Build LSTM Class
- forget gate
- memory gate
- recall gate

In [ ]:
class LSTM(torch.nn.Module):
    def __init__(self, size_in,size_out,size_long,size_short):
        super(LSTM, self).__init__()
        self.size_long = size_long
        self.size_short = size_short
        
        size_z = size_in + size_short
        
        self.forget_gate = Layer(size_z, size_long, torch.sigmoid)
        self.memory_gate = Layer(size_z, size_long, torch.sigmoid)
        
        self.memory_layer = Layer(size_z, size_long, torch.tanh)
        
        self.recall_gate = Layer(size_z, size_short, torch.sigmoid)
        self.recall_layer = Layer(size_long, size_short, torch.tanh)
        self.output_gate = Layer(size_short, size_out, torch.sigmoid)
        
    def Forward(self, X):
        mem_short = torch.zeros(1,self.size_short)
        mem_long = torch.zeros(1, self.size_long)
        
        y_hat = []
        
        for i in range(X.shape[0]):
            X_t = X[[i],:]
            z_t = torch.cat([X_t,mem_short], dim = 1)
            
            mem_long = mem_long*self.forget_gate.Forward(z_t)# z_t are the epoch....
            mem_long = mem_long + (self.memory_gate.Forward(z_t)*self.memory_layer.Forward(z_t))
            mem_short = self.recall_gate.Forward(z_t) + self.recall_layer.Forward(mem_long)
            
            y_hat.append(self.output_gate.Forward(mem_short))
        return torch.cat(y_hat, dim = 0)
            
            #out = self.output_gate.Forward(mem_short)
            #out = torch.argmax(out,dim = 1)
    
    def Generate(self, start, stop, random_factor):
        y_hat = [start]
        
        mem_long = torch.randn([1,self.size_long])*random_factor
        
        mem_short = torch.randn([1,self.size_short])* random_factor
        
        while(y_hat[-1] != stop).any() and len(y_hat) < 30:
            x_t = y_hat[-1]
            z_t = torch.cat([x_t, mem_short], dim=1)
            mem_long = mem_long*self.forget_gate.Forward(z_t)
            mem_long = mem_long +(self.memory_gate.Forward(z_t)*self.memory_layer.Forward(z_t))
            mem_short = self.recall_gate.Forward(z_t)+self.recall_layer.Forward(mem_long)
            out = self.output_gate.Forward(mem_short)
            out = torch.argmax(out, dim =1)
            
            
            #OSTM
            
            y_hat.append(torch.zeros(stop.shape))
            y_hat[-1][0,out] = 1
        return torch.cat(y_hat, dim = 0)